In [47]:
import pandas as pd

## Load Data

In [48]:
disney = pd.read_csv('analysis/disney_pics.csv').drop(['Unnamed: 0', 'name'], axis = 1)
disney.head()

,width,height,mean_hue,mean_saturation,mean_value,mean_entropy,edge_sum
0,67,98,0.272307,0.148943,0.831144,3.778824,0.819642
1,99,99,0.346913,0.500670,0.275656,4.424552,0.585586
2,99,99,0.453854,0.168975,0.484728,4.181699,0.481131
3,99,99,0.351184,0.669260,0.497502,4.214874,0.554411
4,99,99,0.136235,0.926444,0.331088,3.405405,0.365691


In [49]:
hayao = pd.read_csv('analysis/hayao_pics.csv').drop(['Unnamed: 0', 'name'], axis = 1)
hayao.head()

,width,height,mean_hue,mean_saturation,mean_value,mean_entropy,edge_sum
0,67,98,0.281277,0.530815,0.231668,3.895495,0.583236
1,99,99,0.273528,0.405424,0.722306,4.257138,0.662596
2,147,100,0.600145,0.325997,0.711840,3.021263,0.270850
3,147,100,0.507668,0.685993,0.552678,3.737993,0.201171
4,147,100,0.154216,0.728086,0.574216,4.672342,0.498490


In [50]:
# assign labels to each df for classification
label_disney = ['D'] * disney.shape[0]
disney = disney.assign(label=label_disney)
label_hayao = ['H'] * hayao.shape[0]
hayao = hayao.assign(label = label_hayao)

In [51]:
# combine two dfs into one
data = pd.concat([disney,hayao])

# shuffle data
data = data.sample(frac = 1, replace = False).reset_index(drop = True)
data

,width,height,mean_hue,mean_saturation,mean_value,mean_entropy,edge_sum,label
0,99,99,0.590624,0.343552,0.777861,2.119800,0.259793,D
1,99,99,0.829540,0.563975,0.526478,4.478021,0.410721,D
2,99,99,0.706787,0.291783,0.370580,3.424701,0.508114,H
3,99,99,0.565004,0.704920,0.506476,3.437362,0.239284,D
4,147,100,0.390166,0.330693,0.472495,4.811531,0.447779,H
...,...,...,...,...,...,...,...,...
496,99,99,0.417107,0.340975,0.562301,3.758128,0.439126,H
497,99,99,0.323137,0.090322,0.340372,4.987592,0.551519,D
498,99,99,0.001462,0.000068,0.331491,2.146897,0.471220,D
499,99,99,0.208951,0.577108,0.662603,5.024040,0.589367,H


## Data Preprocessing

In [52]:
data.describe()

,width,height,mean_hue,mean_saturation,mean_value,mean_entropy,edge_sum
count,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000
mean,102.033932,99.007984,0.365262,0.436018,0.547438,4.366570,0.537332
std,23.699301,0.583040,0.165117,0.184130,0.182753,0.764381,0.183716
min,67.000000,98.000000,0.001462,0.000068,0.046154,0.801167,0.110002
25%,99.000000,99.000000,0.246940,0.316454,0.415992,4.026364,0.404776
50%,99.000000,99.000000,0.365365,0.443637,0.547617,4.482826,0.513117
75%,99.000000,99.000000,0.491058,0.567867,0.678632,4.907562,0.648178
max,147.000000,100.000000,0.884856,0.926444,0.968928,5.656472,1.262019


In [53]:
X = data.drop('label', axis = 1)
y = data.label

In [54]:
# scale edge_sum with MinMaxScaler
mms = MinMaxScaler(feature_range = (0,1))
edge = X[['edge_sum']]
new_edge = mms.fit_transform(edge)
X[['edge_sum']] = new_edge

In [55]:
# scale all other features except width and height by StandardScaler
scaler = StandardScaler()
scale_feature = X[['mean_hue','mean_saturation','mean_value','mean_entropy']]
fet = scaler.fit_transform(scale_feature)
X[['mean_hue','mean_saturation','mean_value','mean_entropy']] = fet

In [56]:
X.head()

,width,height,mean_hue,mean_saturation,mean_value,mean_entropy,edge_sum
0,99,99,1.366224,-0.502682,1.262102,-2.942269,0.130025
1,99,99,2.814621,0.695622,-0.114804,0.145951,0.261037
2,99,99,2.070451,-0.784120,-0.968713,-1.233429,0.345579
3,99,99,1.210910,1.461855,-0.224364,-1.216849,0.112222
4,147,100,0.150976,-0.572587,-0.410492,0.582701,0.293205


In [57]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

## Model Fitting

In [58]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV



In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [60]:
# define helper function to print out best hyperparameters set
def grid_search_best_hyperparameters(grid, parameters = None):
    
    best_score = grid.best_score_
    best_parameter = grid.best_params_
    print('Model Best Score:', best_score)
    
    
    if parameters != None:
        print('Best Hyperparameters Set:')
        print('    {')
        for parameter_name in sorted(parameters.keys()):
            print('     ' + parameter_name + ':', best_parameter[parameter_name])
        print('    }')
    else:
        return

In [61]:
# Classifiers
rfc = RandomForestClassifier()
gmm = GaussianMixture()
gnb = GaussianNB()
knn = KNeighborsClassifier()

In [62]:
# random forest hyperparameters
rfc_params = {
    
    'n_estimators': [20, 40, 60, 80, 100],
    'max_depth': [5, 10, 15, 20, 25, 30]
    
}
# gaussian mixture hyperparameters
gmm_params = {
    
    'n_components': [1, 2, 3, 4, 5]
    
}

# knn classifier parameters
knn_params = {
    
    'n_neighbors': [3, 5, 7, 9]
    
}

In [ ]:
# fit models
grid_rfc = GridSearchCV(rfc, rfc_params, cv = 5, scoring = 'accuracy')
grid_rfc.fit(x_train, y_train)
print('Random Forest Classifier')
grid_search_best_hyperparameters(grid_rfc, rfc_params)
print('--------------------------------------------')

grid_gmm = GridSearchCV(gmm, gmm_params, cv = 5, scoring = 'accuracy')
grid_gmm.fit(x_train, y_train)
print('Gaussian Mixture Model')
grid_search_best_hyperparameters(grid_gmm, gmm_params)
print('--------------------------------------------')

gnb_cv = cross_val_score(gnb, x_train, y_train, cv = 5, scoring = 'accuracy')
avg = np.max(gnb_cv)
print('Gaussian Naive Bayes Model')
print('Model Average Score:', avg)
print('--------------------------------------------')

grid_knn = GridSearchCV(knn, knn_params, cv = 5, scoring = 'accuracy')
grid_knn.fit(x_train, y_train)
print('K-Nearest Neighbors Model')
grid_search_best_hyperparameters(grid_knn, knn_params)

In [ ]:
print(__doc__)


# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

# X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
#                            random_state=1, n_clusters_per_class=1)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.2, factor=0.5, random_state=1),
            linearly_separable
            ]

figure = plt.figure(figsize=(27, 9))
i = 1
# iterate over datasets
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, split into training and test part
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.3, random_state=42)

    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
#     ax = plt.subplot(len(datasets)*2, len(classifiers)/2 + 1, i)

    if ds_cnt == 0:
        ax.set_title("Input data")
    # Plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
               edgecolors='k')
    # Plot the testing points
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6,
               edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1

    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Plot the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
                   edgecolors='k')
        # Plot the testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
                   edgecolors='k', alpha=0.6)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        if ds_cnt == 0:
            ax.set_title(name)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                size=15, horizontalalignment='right')
        i += 1

plt.tight_layout()
plt.show()